In [151]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense, LSTM
from skimage.metrics import mean_squared_error
import math
import plotly.graph_objects as go

In [152]:
original_crime_df = pd.read_csv("Crime_Data_from_2020_to_Present 2.csv")
print(original_crime_df.head())
print(original_crime_df['Crm Cd'].value_counts()[:10])
new_dataset = original_crime_df[original_crime_df["Crm Cd"].isin([510,624,740,330,310,230,626,440,420,354]) ]
new_dataset.head()

       DR_NO               Date Rptd                DATE OCC  TIME OCC  AREA  \
0   10304468  01/08/2020 12:00:00 AM  01/08/2020 12:00:00 AM      2230     3   
1  190101086  01/02/2020 12:00:00 AM  01/01/2020 12:00:00 AM       330     1   
2  200110444  04/14/2020 12:00:00 AM  02/13/2020 12:00:00 AM      1200     1   
3  191501505  01/01/2020 12:00:00 AM  01/01/2020 12:00:00 AM      1730    15   
4  191921269  01/01/2020 12:00:00 AM  01/01/2020 12:00:00 AM       415    19   

     AREA NAME  Rpt Dist No  Part 1-2  Crm Cd  \
0    Southwest          377         2     624   
1      Central          163         2     624   
2      Central          155         2     845   
3  N Hollywood         1543         2     745   
4      Mission         1998         2     740   

                                         Crm Cd Desc  ... Status  \
0                           BATTERY - SIMPLE ASSAULT  ...     AO   
1                           BATTERY - SIMPLE ASSAULT  ...     IC   
2          SEX OFFEN

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,10304468,01/08/2020 12:00:00 AM,01/08/2020 12:00:00 AM,2230,3,Southwest,377,2,624,BATTERY - SIMPLE ASSAULT,...,AO,Adult Other,624.0,NaN,NaN,NaN,1100 W 39TH PL,NaN,34.0141,-118.2978
1,190101086,01/02/2020 12:00:00 AM,01/01/2020 12:00:00 AM,330,1,Central,163,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,700 S HILL ST,NaN,34.0459,-118.2545
4,191921269,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,415,19,Mission,1998,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740.0,NaN,NaN,NaN,14400 TITUS ST,NaN,34.2198,-118.4468
10,200100509,01/04/2020 12:00:00 AM,01/04/2020 12:00:00 AM,2200,1,Central,192,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,NaN,NaN,NaN,15TH,OLIVE,34.0359,-118.2648
15,211916029,11/26/2021 12:00:00 AM,11/30/2020 12:00:00 AM,730,19,Mission,1916,2,626,INTIMATE PARTNER - SIMPLE ASSAULT,...,IC,Invest Cont,626.0,NaN,NaN,NaN,14200 BERG ST,NaN,34.3055,-118.4439


In [153]:
dataset_510 = original_crime_df[original_crime_df["Crm Cd"].isin([510]) ]
dataset_624 = original_crime_df[original_crime_df["Crm Cd"].isin([624])]

dataset_510

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
33,200117988,09/15/2020 12:00:00 AM,09/03/2020 12:00:00 AM,2000,1,Central,111,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,500 N FIGUEROA ST,NaN,34.0615,-118.2470
78,200104073,01/02/2020 12:00:00 AM,01/02/2020 12:00:00 AM,345,1,Central,143,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,400 S SPRING ST,NaN,34.0488,-118.2500
104,200516527,11/12/2020 12:00:00 AM,11/12/2020 12:00:00 AM,430,5,Harbor,508,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,23400 PRESIDENT AV,NaN,33.8129,-118.3071
105,200609101,04/15/2020 12:00:00 AM,04/12/2020 12:00:00 AM,1300,6,Hollywood,648,1,510,VEHICLE - STOLEN,...,AO,Adult Other,510.0,NaN,NaN,NaN,5700 HOLLYWOOD BL,NaN,34.1017,-118.3136
135,200306227,02/07/2020 12:00:00 AM,02/06/2020 12:00:00 AM,1800,3,Southwest,397,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,43RD ST,MENLO,34.0073,-118.2980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
843455,230915370,10/17/2023 12:00:00 AM,10/17/2023 12:00:00 AM,1000,9,Van Nuys,935,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,CALVERT ST,VAN NUYS BL,34.1816,-118.4487
843465,230713941,08/30/2023 12:00:00 AM,08/30/2023 12:00:00 AM,245,7,Wilshire,745,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,1000 S RIDGELEY DR,NaN,34.0574,-118.3525
843481,231309462,04/20/2023 12:00:00 AM,04/20/2023 12:00:00 AM,900,13,Newton,1343,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,900 E JEFFERSON BL,NaN,34.0136,-118.2611
843491,230406516,03/19/2023 12:00:00 AM,03/19/2023 12:00:00 AM,122,4,Hollenbeck,488,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,3200 GARNET ST,NaN,34.0248,-118.2076


In [154]:
dataset_510['Date Rptd'] = pd.to_datetime(dataset_510['Date Rptd'])
dataset_510['DATE OCC'] = pd.to_datetime(dataset_510['DATE OCC'])

dataset_510.sort_values(by="DATE OCC")

/var/folders/8g/z0__2mts0rnbjh3kq8zsfz2w0000gn/T/ipykernel_6574/214316822.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/8g/z0__2mts0rnbjh3kq8zsfz2w0000gn/T/ipykernel_6574/214316822.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
189498,201205133,2020-01-17,2020-01-01,1400,12,77th Street,1256,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,900 W 74TH ST,NaN,33.9727,-118.2893
142704,201804050,2020-01-02,2020-01-01,1600,18,Southeast,1863,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,IMPERIAL HY,AVALON BL,33.9311,-118.2652
59855,200313800,2020-07-07,2020-01-01,800,3,Southwest,392,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,3000 W 48TH ST,NaN,34.0000,-118.3269
1592,200204391,2020-01-09,2020-01-01,2300,2,Rampart,219,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,900 E EDGEWARE RD,NaN,34.0698,-118.2528
30211,200207801,2020-03-16,2020-01-01,1500,2,Rampart,279,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,900 S UNION AV,NaN,34.0508,-118.2731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
800095,230515996,2023-11-20,2023-11-20,510,5,Harbor,567,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,2000 S PACIFIC AV,NaN,33.7261,-118.2880
695708,231717853,2023-11-20,2023-11-20,1120,17,Devonshire,1785,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,8800 DARBY AV,NaN,34.2308,-118.5336
677441,231223417,2023-11-20,2023-11-20,300,12,77th Street,1239,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,VERMONT,60TH ST,33.9855,-118.2915
842487,232018061,2023-11-20,2023-11-20,200,20,Olympic,2004,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,300 N OXFORD AV,NaN,34.0763,-118.3079


In [155]:
df1 = dataset_510['DATE OCC'].dt.date.value_counts().sort_index().reset_index()
df1.columns = ['DATE','Count']

df1['DATE'] = pd.to_datetime(df1['DATE']) - pd.to_timedelta(7, unit='d')
df1= df1.groupby([pd.Grouper(key='DATE', freq='W')])['Count'].sum()

df1 = pd.DataFrame(data=df1.values, columns=['Count'])
df1 = df1.values
df1 = df1.astype('int32')
df1

array([[196],
       [346],
       [308],
       [317],
       [343],
       [312],
       [310],
       [307],
       [306],
       [322],
       [337],
       [307],
       [363],
       [413],
       [416],
       [412],
       [443],
       [394],
       [450],
       [414],
       [435],
       [400],
       [460],
       [468],
       [495],
       [513],
       [523],
       [432],
       [439],
       [439],
       [441],
       [415],
       [421],
       [382],
       [430],
       [386],
       [375],
       [389],
       [372],
       [388],
       [416],
       [383],
       [442],
       [412],
       [411],
       [409],
       [425],
       [398],
       [414],
       [403],
       [393],
       [390],
       [421],
       [408],
       [412],
       [448],
       [435],
       [488],
       [460],
       [470],
       [415],
       [440],
       [412],
       [395],
       [397],
       [409],
       [386],
       [400],
       [441],
       [463],
       [409],
      

In [156]:
def formatNumber(n):
    return '{:,}'.format(n).replace(","," ")
def highlight_diag(df):
    a = np.full(df.shape, '', dtype='<U24')
    np.fill_diagonal(a, 'background-color: yellow')
    return pd.DataFrame(a, index=df.index, columns=df.columns)
def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [157]:
list_graphs = []
graph_1 = go.Scatter(x=list(range(0,len(df1))),y=df1.flatten(),name="Dataset")
list_graphs.append(graph_1)
average = moving_average(df1)
graph_2 = go.Scatter(x=list(range(0,len(df1))),y=average,name="Average")
list_graphs.append(graph_2)
fig = go.Figure(data=list_graphs)
fig.update_layout(width=1024,height=500,title="Dataset of crimes in ",xaxis=dict(title='Weeks'),yaxis=dict(title='Number of crimes'))

fig.show()

In [158]:
scaler = MinMaxScaler()
df1 = scaler.fit_transform(df1)

In [159]:
df1
train_size = int(len(df1) * 0.7)
print(train_size)
test_size = len(df1) - train_size
print(test_size)
train, test = df1[0:train_size,:], df1[train_size:len(df1),:]

142
62


In [160]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i+look_back),0]
        dataX.append(a)
        dataY.append(dataset[i + look_back,0])
    return np.array(dataX), np.array(dataY)

In [161]:
look_back = 2
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [162]:
trainX.shape, trainY.shape, testX.shape, testY.shape

((140, 2), (140,), (60, 2), (60,))

In [163]:
trainX = np.reshape(trainX, (trainX.shape[0],1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0],1, testX.shape[1]))

In [164]:
model = Sequential()
model.add(LSTM(8,activation='tanh',return_sequences=True, input_shape=(1,look_back)))
model.add(LSTM(4,activation='tanh',return_sequences=False, input_shape=(1,look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='rmsprop')
model.fit(trainX, trainY, epochs=500, batch_size=4, verbose=1)

Epoch 1/500
35/35 [==============================] - 1s 952us/step - loss: 0.5423
Epoch 2/500
35/35 [==============================] - 0s 702us/step - loss: 0.3879
Epoch 3/500
35/35 [==============================] - 0s 741us/step - loss: 0.2405
Epoch 4/500
35/35 [==============================] - 0s 569us/step - loss: 0.1131
Epoch 5/500
35/35 [==============================] - 0s 548us/step - loss: 0.0310
Epoch 6/500
35/35 [==============================] - 0s 620us/step - loss: 0.0061
Epoch 7/500
35/35 [==============================] - 0s 630us/step - loss: 0.0051
Epoch 8/500
35/35 [==============================] - 0s 538us/step - loss: 0.0049
Epoch 9/500
35/35 [==============================] - 0s 555us/step - loss: 0.0048
Epoch 10/500
35/35 [==============================] - 0s 595us/step - loss: 0.0047
Epoch 11/500
35/35 [==============================] - 0s 572us/step - loss: 0.0045
Epoch 12/500
35/35 [==============================] - 0s 665us/step - loss: 0.0044
Epoch 13/500


In [165]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

2/2 [==============================] - 0s 1ms/step


In [166]:
trainPredict.shape, trainY.shape
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [167]:
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

results= [{ "Models": "LSTM",
            "RMSE": testScore}]

results = pd.DataFrame(results,columns=['Models','RMSE'])

Train Score: 27.34 RMSE
Test Score: 66.45 RMSE


In [168]:
trainPredictPlot = np.empty_like(df1)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

In [169]:
testPredictPlot = np.empty_like(df1)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+look_back+1:len(df1)-1, :] = testPredict

In [170]:
listGraph = []
listGraph.append(go.Scatter(x=list(range(0,len(df1))),y=scaler.inverse_transform(df1).flatten(),name="Original dataset"))
listGraph.append(go.Scatter(x=list(range(0,len(trainPredictPlot))),y=trainPredictPlot.flatten(),name="Predict with Train set"))
listGraph.append(go.Scatter(x=list(range(0,len(testPredictPlot))),y=testPredictPlot.flatten(),name="Predict with Test set"))

fig = go.Figure(data=listGraph)
fig.update_layout(width=1024,height=500,title="[LSTM] Prediction for Stolen Vehicle (Code 510)",xaxis=dict(title='Weeks'),yaxis=dict(title='Number of crimes'))

fig.show()

In [171]:
dataset_624['Date Rptd'] = pd.to_datetime(dataset_624['Date Rptd'])
dataset_624['DATE OCC'] = pd.to_datetime(dataset_624['DATE OCC'])

dataset_624.sort_values(by="DATE OCC")

/var/folders/8g/z0__2mts0rnbjh3kq8zsfz2w0000gn/T/ipykernel_6574/2714906879.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/8g/z0__2mts0rnbjh3kq8zsfz2w0000gn/T/ipykernel_6574/2714906879.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
101031,201904016,2020-01-01,2020-01-01,300,19,Mission,1982,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,15200 ACRE ST,NaN,34.2300,-118.4644
129754,201704020,2020-01-01,2020-01-01,15,17,Devonshire,1785,2,624,BATTERY - SIMPLE ASSAULT,...,AO,Adult Other,624.0,NaN,NaN,NaN,18700 SUNBURST ST,NaN,34.2346,-118.5403
44085,200204049,2020-01-01,2020-01-01,1830,2,Rampart,269,2,624,BATTERY - SIMPLE ASSAULT,...,AO,Adult Other,624.0,NaN,NaN,NaN,1400 JAMES M WOOD BL,NaN,34.0508,-118.2683
180309,201704356,2020-01-01,2020-01-01,1600,17,Devonshire,1768,2,624,BATTERY - SIMPLE ASSAULT,...,AO,Adult Other,624.0,NaN,NaN,NaN,9800 WISH AV,NaN,34.2484,-118.5079
130229,202004026,2020-01-01,2020-01-01,430,20,Olympic,2002,2,624,BATTERY - SIMPLE ASSAULT,...,AO,Adult Other,624.0,NaN,NaN,NaN,600 N SAINT ANDREWS PL,NaN,34.0817,-118.3112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732158,230221254,2023-11-20,2023-11-20,200,2,Rampart,257,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,1200 WILSHIRE BL,NaN,34.0530,-118.2649
762894,231320891,2023-11-20,2023-11-20,100,13,Newton,1343,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,600 E 38TH ST,NaN,34.0120,-118.2653
841814,231916845,2023-11-20,2023-11-20,1515,19,Mission,1998,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,VAN NUYS,ROSCOE,34.2215,-118.4488
690389,230124448,2023-11-20,2023-11-20,230,1,Central,156,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,500 S SAN JULIAN ST,NaN,34.0446,-118.2450


In [172]:
df1 = dataset_624['DATE OCC'].dt.date.value_counts().sort_index().reset_index()
df1.columns = ['DATE','Count']

df1['DATE'] = pd.to_datetime(df1['DATE']) - pd.to_timedelta(7, unit='d')
df1= df1.groupby([pd.Grouper(key='DATE', freq='W')])['Count'].sum()

df1 = pd.DataFrame(data=df1.values, columns=['Count'])
df1 = df1.values
df1 = df1.astype('int32')

In [173]:
list_graphs = []
graph_1 = go.Scatter(x=list(range(0,len(df1))),y=df1.flatten(),name="Dataset")
list_graphs.append(graph_1)
average = moving_average(df1)
graph_2 = go.Scatter(x=list(range(0,len(df1))),y=average,name="Average")
list_graphs.append(graph_2)
fig = go.Figure(data=list_graphs)
fig.update_layout(width=1024,height=500,title="Dataset of crimes in ",xaxis=dict(title='Weeks'),yaxis=dict(title='Number of crimes'))

fig.show()

In [174]:
scaler = MinMaxScaler()
df1 = scaler.fit_transform(df1)

In [175]:
train_size = int(len(df1) * 0.7)
print(train_size)
test_size = len(df1) - train_size
print(test_size)
train, test = df1[0:train_size,:], df1[train_size:len(df1),:]

142
62


In [176]:
look_back = 2
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [177]:
trainX.shape, trainY.shape, testX.shape, testY.shape

((140, 2), (140,), (60, 2), (60,))

In [178]:
trainX = np.reshape(trainX, (trainX.shape[0],1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0],1, testX.shape[1]))

In [179]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

model = Sequential()
model.add(LSTM(16,activation='tanh',return_sequences=True, input_shape=(1,look_back)))
model.add(LSTM(8,activation='tanh',return_sequences=True, input_shape=(1,look_back)))
model.add(LSTM(4,activation='tanh',return_sequences=False, input_shape=(1,look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['accuracy'])
model.fit(trainX, trainY, epochs=500, batch_size=4, verbose=1)

Epoch 1/500
35/35 [==============================] - 2s 1ms/step - loss: 0.4841 - accuracy: 0.0000e+00
Epoch 2/500
35/35 [==============================] - 0s 1ms/step - loss: 0.3396 - accuracy: 0.0000e+00
Epoch 3/500
35/35 [==============================] - 0s 1ms/step - loss: 0.1778 - accuracy: 0.0000e+00
Epoch 4/500
35/35 [==============================] - 0s 833us/step - loss: 0.0466 - accuracy: 0.0000e+00
Epoch 5/500
35/35 [==============================] - 0s 843us/step - loss: 0.0071 - accuracy: 0.0000e+00
Epoch 6/500
35/35 [==============================] - 0s 816us/step - loss: 0.0065 - accuracy: 0.0000e+00
Epoch 7/500
35/35 [==============================] - 0s 802us/step - loss: 0.0062 - accuracy: 0.0000e+00
Epoch 8/500
35/35 [==============================] - 0s 767us/step - loss: 0.0063 - accuracy: 0.0000e+00
Epoch 9/500
35/35 [==============================] - 0s 804us/step - loss: 0.0062 - accuracy: 0.0000e+00
Epoch 10/500
35/35 [==============================] - 0s 790u

In [180]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

2/2 [==============================] - 0s 2ms/step


In [181]:
trainPredict.shape, trainY.shape
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [182]:
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 29.05 RMSE
Test Score: 53.30 RMSE


In [183]:
trainPredictPlot = np.empty_like(df1)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

In [184]:
testPredictPlot = np.empty_like(df1)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict) + look_back + 1:len(df1) - 1, :] = testPredict

In [185]:
listGraph = []
listGraph.append(go.Scatter(x=list(range(0,len(df1))),y=scaler.inverse_transform(df1).flatten(),name="Original dataset"))
listGraph.append(go.Scatter(x=list(range(0,len(trainPredictPlot))),y=trainPredictPlot.flatten(),name="Predict with Train set"))
listGraph.append(go.Scatter(x=list(range(0,len(testPredictPlot))),y=testPredictPlot.flatten(),name="Predict with Test set"))

fig = go.Figure(data=listGraph)
fig.update_layout(width=1024,height=500,title="[LSTM] Prediction for Battery - Simple Assault (Code 624)",xaxis=dict(title='Weeks'),yaxis=dict(title='Number of crimes'))

fig.show()

In [186]:
df3 = dataset_510['DATE OCC'].dt.date.value_counts().sort_index().reset_index()
df3.columns = ['DATE','Count']

df3['DATE'] = pd.to_datetime(df3['DATE']) - pd.to_timedelta(7, unit='d')
df3= df3.groupby([pd.Grouper(key='DATE', freq='W')])['Count'].sum()

In [187]:
df3 = pd.DataFrame(data=df3.values, columns=['Count'])
df3 = df3.values
df3 = df3.astype('int32')

In [188]:
train_size = int(len(df3) * 0.7)
print(train_size)
test_size = len(df3) - train_size
print(test_size)
train, test = df3[0:train_size,:], df3[train_size:len(df3),:]

142
62


In [189]:
from pmdarima.arima import auto_arima
predictions=[]

model = auto_arima(train,m=12,start_p=0,start_q=3, max_order=5,
                   error_action='ignore',test='adf',seasonal=True,
                   trace=True,stepwise=True)
model_fit = model.fit(train)
output = model_fit.predict(n_periods=len(test))

Performing stepwise search to minimize aic
 ARIMA(0,1,3)(1,0,1)[12] intercept   : AIC=1371.216, Time=0.86 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   : AIC=1393.165, Time=0.01 sec
 ARIMA(1,1,0)(1,0,0)[12] intercept   : AIC=1369.702, Time=0.11 sec
 ARIMA(0,1,1)(0,0,1)[12] intercept   : AIC=1367.379, Time=0.13 sec
 ARIMA(0,1,0)(0,0,0)[12]             : AIC=1391.796, Time=0.01 sec
 ARIMA(0,1,1)(0,0,0)[12] intercept   : AIC=1365.495, Time=0.04 sec
 ARIMA(0,1,1)(1,0,0)[12] intercept   : AIC=1367.383, Time=0.17 sec
 ARIMA(0,1,1)(1,0,1)[12] intercept   : AIC=1369.344, Time=0.30 sec
 ARIMA(1,1,1)(0,0,0)[12] intercept   : AIC=1366.041, Time=0.04 sec
 ARIMA(0,1,2)(0,0,0)[12] intercept   : AIC=1365.468, Time=0.03 sec
 ARIMA(0,1,2)(1,0,0)[12] intercept   : AIC=1367.380, Time=0.50 sec
 ARIMA(0,1,2)(0,0,1)[12] intercept   : AIC=1367.378, Time=0.32 sec
 ARIMA(0,1,2)(1,0,1)[12] intercept   : AIC=1369.380, Time=0.28 sec
 ARIMA(1,1,2)(0,0,0)[12] intercept   : AIC=1367.437, Time=0.08 sec
 ARIMA(0,1,3)(0,0,0

In [190]:
actual =[]
for i in test:
    actual.append(i[0])
output, actual = np.array(output),np.array(actual)
testScore = math.sqrt(mean_squared_error(actual, output))
print("RMSE For ARIMA model :",testScore)
new_row = { "Models": "ARIMA",
            "RMSE": testScore}
results = results.append(new_row,ignore_index=True)

RMSE For ARIMA model : 78.60141533983035


/var/folders/8g/z0__2mts0rnbjh3kq8zsfz2w0000gn/T/ipykernel_6574/3387129174.py:9: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [191]:
df4 = dataset_510['DATE OCC'].dt.date.value_counts().sort_index().reset_index()
df4.columns = ['ds','y']

df4['ds'] = pd.to_datetime(df4['ds']) - pd.to_timedelta(7, unit='d')
df4= df4.groupby([pd.Grouper(key='ds', freq='W')])['y'].sum()

In [192]:
train_size = int(len(df4) * 0.7)
print(train_size)
test_size = len(df4) - train_size
print(test_size)
train, test = df4[0:train_size], df4[train_size:len(df4)]

142
62


In [193]:
prophet_df = train.reset_index()
prophet_df .head()
prophet_df = prophet_df.rename(columns= {'Date':'ds','Crime Count':'y'})
prophet_df.columns

Index(['ds', 'y'], dtype='object')

In [194]:
from prophet import Prophet
prophet_df = prophet_df.rename(columns= {'Date':'ds','Crime Count':'y'})
m = Prophet()
m.fit(prophet_df)
future = m.make_future_dataframe(periods=len(test),freq='W',include_history = False)
forecast = m.predict(future)
forecast.head()

02:40:35 - cmdstanpy - INFO - Chain [1] start processing
02:40:35 - cmdstanpy - INFO - Chain [1] done processing


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2022-09-18,513.961920,462.955832,562.103352,513.961920,513.961920,0.515929,0.515929,0.515929,0.515929,0.515929,0.515929,0.0,0.0,0.0,514.477849
1,2022-09-25,514.929855,478.356691,574.324080,514.911012,514.942543,9.889381,9.889381,9.889381,9.889381,9.889381,9.889381,0.0,0.0,0.0,524.819236
2,2022-10-02,515.897791,484.065253,582.992306,515.846813,515.933005,16.075943,16.075943,16.075943,16.075943,16.075943,16.075943,0.0,0.0,0.0,531.973733
3,2022-10-09,516.865726,488.606039,584.080095,516.766765,516.947621,21.435374,21.435374,21.435374,21.435374,21.435374,21.435374,0.0,0.0,0.0,538.301100
4,2022-10-16,517.833661,498.818612,593.735352,517.687158,517.973896,27.780817,27.780817,27.780817,27.780817,27.780817,27.780817,0.0,0.0,0.0,545.614478


In [195]:
forecast_df = pd.DataFrame(forecast)
forecast_df=forecast_df[['ds','yhat']]
forecast_df=forecast_df.set_index('ds')
forecast_df.count()

yhat    62
dtype: int64

In [196]:
forecast_df.head()

,yhat
ds,
2022-09-18,514.477849
2022-09-25,524.819236
2022-10-02,531.973733
2022-10-09,538.301100
2022-10-16,545.614478


In [197]:
df4 = df4.to_frame()

In [198]:
RMSE = ((df4.y - forecast_df.yhat) ** 2).mean() ** .5
new_row = { "Models": "FB Prophet",
            "RMSE": RMSE}
results = results.append(new_row,ignore_index=True)
print(RMSE)

119.25189922493561


/var/folders/8g/z0__2mts0rnbjh3kq8zsfz2w0000gn/T/ipykernel_6574/3201360177.py:4: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [199]:
display(results)

,Models,RMSE
0,LSTM,66.452827
1,ARIMA,78.601415
2,FB Prophet,119.251899
